In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import Models.Corn_USA_Yield as cy
import Utilities.GLOBAL as GV
import Utilities.Modeling as um
from datetime import datetime as dt

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
scope = cy.Define_Scope()

In [11]:
%%timeit
# raw_data = cy.Get_Data(scope)
raw_data = cy.Get_Data_fast(scope)

3.12 s ± 180 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
milestones = cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units='in', temp_units='F')

In [6]:
train_df = cy.Build_Train_DF(raw_data, milestones, intervals, train_DF_instr)
model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
# print(model.summary())

In [7]:
instructions=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F')

In [10]:
%%time
pred_df = cy.Build_Pred_DF(raw_data,milestones,instructions,GV.CUR_YEAR, dt(2022,5,1))
yields = model.predict(pred_df[model.params.index]).values
yields

Days to calculate: 64
Wall time: 1.84 s


array([180.73167811, 180.74602553, 180.60749826, 180.60719554,
       180.62094966, 180.52838836, 180.53550842, 180.54267476,
       180.58044888, 180.61405796, 180.68881358, 180.69047557,
       180.69378043, 180.72288364, 175.89984533, 175.91227307,
       175.9421786 , 175.94061745, 175.94394937, 175.93288782,
       175.93585574, 175.91576668, 175.94356629, 175.95585582,
       175.94045452, 175.91340173, 175.90105986, 175.9210688 ,
       175.93852173, 175.93193525, 175.90702003, 175.90813881,
       175.92047273, 175.94139529, 175.95237749, 175.95684856,
       175.94557591, 175.9141649 , 175.8873087 , 175.88737491,
       175.91562902, 175.93707575, 175.94543721, 175.95024043,
       175.95433469, 175.95387861, 175.95670755, 175.95698539,
       175.94784487, 175.92168952, 175.73885658, 175.43699721,
       175.21533476, 174.95175896, 174.66032732, 174.50082306,
       174.62405122, 174.49168151, 174.28867071, 174.00537785,
       173.61052385, 173.49015907, 173.51653273, 173.31

In [9]:
pred_df

,const,Trend,Yield,Planted pct on May 15th,Planting Prec,Planting Prec Squared,Jul Aug Prec,Jul Aug Prec Squared,Prec Interaction,Pollination SDD,Regular SDD
2022,1.0,2022,NaN,72.289575,9.147167,83.670672,9.889243,97.797123,90.458559,58.816266,64.895429
2022,1.0,2022,NaN,72.289575,9.095832,82.734158,9.889243,97.797123,89.950890,58.816266,64.895429
2022,1.0,2022,NaN,72.289575,9.443384,89.177494,9.889243,97.797123,93.387913,58.816266,64.895429
2022,1.0,2022,NaN,72.289575,9.443946,89.188111,9.889243,97.797123,93.393472,58.816266,64.895429
2022,1.0,2022,NaN,72.289575,9.417911,88.697050,9.889243,97.797123,93.136010,58.816266,64.895429
...,...,...,...,...,...,...,...,...,...,...,...
2022,1.0,2022,NaN,49.000000,8.331282,69.410254,9.222556,85.055545,76.835714,58.816266,75.125540
2022,1.0,2022,NaN,49.000000,8.331282,69.410254,9.103205,82.868335,75.841362,58.816266,76.910997
2022,1.0,2022,NaN,49.000000,8.331282,69.410254,9.076111,82.375785,75.615634,58.816266,77.720823
2022,1.0,2022,NaN,49.000000,8.331282,69.410254,9.051719,81.933608,75.412416,58.816266,76.496585


In [ ]:
milestones['15th_May_pct_planted'].tail()

In [ ]:
milestones.keys()

In [ ]:
test={}

a={'a':1}
b={'b':2}

In [ ]:
test.update(a)
test.update(b)
test